In [1]:
!pip install transformers

In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# from AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-1.3B")

model = AutoModelForSeq2SeqLM.from_pretrained("results/checkpoint-504")
model.to("cuda")

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  4.59it/s]


In [3]:
import pandas as pd

df = pd.read_csv('opus_eng_to_tgl.csv')
val = df[df['Split'] == 'test']
source = list(val['Filipino'])
target = list(val['English'])

source_sentences = [sent.strip() for sent in source]
target_sentences = [sent.strip() for sent in target]

print(source_sentences[0])
print(target_sentences[0])

May tiwala ako sa iyo. Alam mo, inosente ka.
I trust you, you know what, and you're innocent.


# Translation Predictions

In [4]:
def create_prompt(source_lang,
                  target_lang,
                  fuzzy_sources,
                  fuzzy_targets,
                  new_sources,
                  one_shot=True
                  ):

  prompts = []

  if one_shot:
    for fuzzy_src, fuzzy_tgt, new_src in zip(fuzzy_sources, fuzzy_targets, new_sources):
      fuzzy_src = source_lang + ": " + fuzzy_src
      fuzzy_tgt = target_lang + ": " + fuzzy_tgt
      new_src = source_lang + ": " + new_src
      segment = fuzzy_src + "\n" + fuzzy_tgt + "\n" + new_src + "\n" + target_lang + ":"
      prompts.append(segment)
  else:
    for new_src in new_sources:
      new_src = source_lang + ": " + new_src
      segment = new_src + "\n" + target_lang + ":"
      prompts.append(segment)

  return prompts

In [5]:
import random


shots = df[df['Split'] == 'shots']
shots_source = list(val['Filipino'])
shots_target = list(val['English'])

fuzzy_shots = []

for sent in source_sentences:
    idx = random.randint(0, len(shots_source) - 1)
    fuzzy_shots.append([shots_source[idx], shots_target[idx]])

fuzzy_source_sentences, fuzzy_target_prefixes = list(zip(*fuzzy_shots))
online_source_sentences = source_sentences

In [6]:
source_lang = "Filipino"
target_lang = "English"

# Create prompts
# Set one_shot=True to create a one-shot prompts

zero_shot_prompts = create_prompt(source_lang,
                        target_lang,
                        fuzzy_source_sentences,
                        fuzzy_target_prefixes,
                        online_source_sentences,
                        one_shot=False
                        )
one_shot_prompts = create_prompt(source_lang,
                        target_lang,
                        fuzzy_source_sentences,
                        fuzzy_target_prefixes,
                        online_source_sentences,
                        one_shot=True
                        )

In [22]:
model_inputs = tokenizer("English: And I can't wait to tear your clothes off and get you back to bed.", return_tensors='pt').to("cuda")
generated_ids = model.generate(**model_inputs)
print(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

Filipino: At hindi ako makapaghintay upang hilahin ang iyong damit off at makakuha ka bumalik sa kama.
